<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9ec1dbd45873481a42af62c86d1eae8bc33113187a575f093880a5ccbfe639bb
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-19 12:03:30
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.39 C
-------------------
Today PnL: -62.91 K (-0.45%)
Current PnL: -29.87 L (-19.05%)
CY Booked + Current PnL: -15.58 L (-9.94%)
-------------------
Total profit:  1.66 L
Total loss:  -31.53 L
-------------------
Total Booked + Current PnL: 11.40 L (8.78%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 95.22 L (68.4%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.38,15.57,5.04,21.40,15460.0,41325.0,306751.0,7.22,60.0,X-LC,6.79,14.0,2.67,2.20,22.88,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.13,5.85,12.63,19.23,20115.0,8806.0,159266.0,24.27,68.0,L-LC,10.45,182.0,0.44,1.14,28.37,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-2.19,-24.10,31.87,0.09,24378.0,-24286.0,76491.0,77.59,30.0,M-SC,4.44,253.0,-1.00,0.55,0.49,OX40N,NTT,DURABLES
49,MASFIN,398.61,0.53,-4.16,27.35,22.05,25682.0,-4080.0,93900.0,-17.34,47.0,H-SC,5.19,168.0,-0.16,0.67,36.49,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-3.49,-15.24,18.00,0.02,26632.0,-26602.0,147957.0,125.45,49.0,M-SC,9.37,240.0,-1.00,1.06,28.70,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,8.25,-1.64,110.16,106.72,133643.0,-2019.0,121317.0,-51.39,60.0,H-SC,8.30,138.0,-0.02,0.87,21.36,AR,ATH,MISC
46,JSWINFRA,342.00,6.38,-6.67,24.75,16.43,51228.0,-14791.0,206983.0,-21.02,50.0,X-MC,7.51,66.0,-0.29,1.49,22.15,X40N,NTT,REALTY
73,TCS,4311.58,2.27,-11.83,34.46,18.55,114923.0,-44740.0,333497.0,-23.95,54.0,X-LC,1.68,3.0,-0.39,2.40,13.46,X40,ATH,IT
22,DIXON,18931.72,1.87,-24.20,73.16,31.26,127977.0,-55840.0,174928.0,-64.60,27.0,X-MC,22.78,56.0,-0.44,1.26,1.87,X40N,ATH,IT
56,RAJOOENG,143.10,1.39,-35.76,117.31,39.61,77249.0,-36650.0,65850.0,-56.61,46.0,H-SC,30.24,136.0,-0.47,0.47,6.59,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-7.10,1.90,69.05,72.27,119231.0,3225.0,172673.0,-12.43,37.0,M-LC,5.78,101.0,0.03,1.24,8.20,XR,NTT,IT
55,RAJESHEXPO,518.0,-3.65,-67.74,210.18,0.07,93015.0,-92922.0,44255.0,1522.47,38.0,L-SC,1.26,269.0,-1.00,0.32,10.06,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.0,-3.49,-15.24,18.00,0.02,26632.0,-26602.0,147957.0,125.45,49.0,M-SC,9.37,240.0,-1.00,1.06,28.70,OX40N,NTT,PAINTS
59,RELIANCE,1952.0,-3.29,-9.50,38.45,25.30,69932.0,-19086.0,181877.0,-12.25,25.0,X-LC,6.55,26.0,-0.27,1.31,21.83,XY25,BTT,REFINERIES
58,RELAXO,1176.0,-3.17,-51.30,212.77,52.31,150403.0,-74472.0,70688.0,-48.12,23.0,X-SC,8.47,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.3,0.43,20.22,20.74,41259.0,880.0,204050.0,33.49,57.0,M-MC,5.16,189.0,0.02,1.47,11.09,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.08,-3.28,114.96,107.91,164166.0,-4841.0,142803.0,-21.36,48.0,M-SC,9.96,220.0,-0.03,1.03,6.48,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-3.49,-15.24,18.00,0.02,26632.0,-26602.0,147957.0,125.45,49.0,M-SC,9.37,240.0,-1.00,1.06,28.70,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-1.25,-20.22,65.50,32.03,68889.0,-26661.0,105174.0,-13.69,54.0,H-SC,6.42,157.0,-0.39,0.76,17.80,OX40N,NTT,IT
47,KANSAINER,340.00,0.24,-22.57,46.56,13.47,97213.0,-60876.0,208791.0,-68.12,47.0,H-SC,4.08,159.0,-0.63,1.50,6.16,XY24,NTT,PAINTS
66,SIS,528.00,0.30,-23.97,59.49,21.26,50220.0,-26614.0,84418.0,1991.71,48.0,H-SC,4.76,163.0,-0.53,0.61,14.06,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.30,0.43,20.22,20.74,41259.0,880.0,204050.0,33.49,57.0,M-MC,5.16,189.0,0.02,1.47,11.09,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.13,5.85,12.63,19.23,20115.0,8806.0,159266.0,24.27,68.0,L-LC,10.45,182.0,0.44,1.14,28.37,XY25,ATH,MINING
84,WIPRO,420.00,-7.10,1.90,69.05,72.27,119231.0,3225.0,172673.0,-12.43,37.0,M-LC,5.78,101.0,0.03,1.24,8.20,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-7.10,1.90,69.05,72.27,119231.0,3225.0,172673.0,-12.43,37.0,M-LC,5.78,101.0,0.03,1.24,8.20,XR,NTT,IT
38,INDIAMART,4810.62,8.25,-1.64,110.16,106.72,133643.0,-2019.0,121317.0,-51.39,60.0,H-SC,8.30,138.0,-0.02,0.87,21.36,AR,ATH,MISC
25,FINCABLES,1641.55,-2.08,-3.28,114.96,107.91,164166.0,-4841.0,142803.0,-21.36,48.0,M-SC,9.96,220.0,-0.03,1.03,6.48,OX40N,ATH,CABLES
37,IEX,219.00,-0.06,-5.47,57.23,48.63,108174.0,-10937.0,189017.0,-37.19,48.0,H-SC,13.29,137.0,-0.10,1.36,5.51,XR,NTT,MISC
49,MASFIN,398.61,0.53,-4.16,27.35,22.05,25682.0,-4080.0,93900.0,-17.34,47.0,H-SC,5.19,168.0,-0.16,0.67,36.49,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.32,-19.06,36.87,10.78,71718.0,-45801.0,194517.0,-51.78,14.0,X-LC,10.83,1.0,-0.64,1.40,0.32,X40,NTT,FMCG
7,AWL,485.0,-1.03,-31.86,130.69,57.19,290425.0,-103903.0,222224.0,-67.84,16.0,X-MC,11.77,58.0,-0.36,1.60,0.00,XY24,NTT,FMCG
11,BATAINDIA,2096.0,-1.09,-42.40,134.73,35.20,99855.0,-54555.0,74115.0,-2.82,22.0,X-SC,17.79,93.0,-0.55,0.53,0.00,X40,NTT,FOOTWEAR
58,RELAXO,1176.0,-3.17,-51.30,212.77,52.31,150403.0,-74472.0,70688.0,-48.12,23.0,X-SC,8.47,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
59,RELIANCE,1952.0,-3.29,-9.50,38.45,25.30,69932.0,-19086.0,181877.0,-12.25,25.0,X-LC,6.55,26.0,-0.27,1.31,21.83,XY25,BTT,REFINERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.58,2.27,-11.83,34.46,18.55,114923.0,-44740.0,333497.0,-23.95,54.0,X-LC,1.68,3.0,-0.39,2.40,13.46,X40,ATH,IT
45,JIOFIN,387.00,-0.88,-11.11,40.04,24.48,95159.0,-29704.0,237661.0,-12.62,31.0,X-LC,1.84,37.0,-0.31,1.71,37.75,XY24,BTT,FINANCE
78,UNITDSPR,1644.00,-1.03,-1.95,23.16,20.77,53172.0,-4558.0,229586.0,-10.48,36.0,X-MC,2.05,70.0,-0.09,1.65,4.84,X40N,NTT,BREWERIES
20,DABUR,735.00,-0.42,0.18,43.44,43.70,107954.0,446.0,248514.0,-8.14,52.0,X-MC,2.11,73.0,0.00,1.79,15.26,XY24,BTT,FMCG
35,ICICIGI,2252.93,-0.29,-1.22,20.82,19.34,41152.0,-2451.0,197658.0,-21.92,34.0,X-MC,2.57,71.0,-0.06,1.42,14.52,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.03,-31.86,130.69,57.19,290425.0,-103903.0,222224.0,-67.84,16.0,X-MC,11.77,58.0,-0.36,1.60,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.99,-21.64,98.40,55.47,187594.0,-52642.0,190644.0,-30.61,28.0,X-MC,16.09,63.0,-0.28,1.37,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.09,-42.40,134.73,35.20,99855.0,-54555.0,74115.0,-2.82,22.0,X-SC,17.79,93.0,-0.55,0.53,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.67,-8.85,47.22,34.19,91901.0,-18896.0,194624.0,-35.86,26.0,X-MC,4.89,60.0,-0.21,1.40,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-3.17,-51.30,212.77,52.31,150403.0,-74472.0,70688.0,-48.12,23.0,X-SC,8.47,92.0,-0.50,0.51,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.85,-31.19,107.59,42.84,48126.0,-20275.0,44731.0,-54.82,39.0,X-SC,23.79,83.0,-0.42,0.32,1.20,XY24,NTT,MISC
58,RELAXO,1176.00,-3.17,-51.30,212.77,52.31,150403.0,-74472.0,70688.0,-48.12,23.0,X-SC,8.47,92.0,-0.50,0.51,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.09,-42.40,134.73,35.20,99855.0,-54555.0,74115.0,-2.82,22.0,X-SC,17.79,93.0,-0.55,0.53,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.28,-5.69,31.56,24.08,37786.0,-7223.0,119727.0,-13.35,38.0,X-SC,6.01,91.0,-0.19,0.86,12.81,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.65,-17.13,72.94,43.32,98454.0,-27892.0,134980.0,-28.80,49.0,X-SC,8.12,90.0,-0.28,0.97,3.64,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.32,-19.06,36.87,10.78,71718.0,-45801.0,194517.0,-51.78,14.0,X-LC,10.83,1.0,-0.64,1.40,0.32,X40,NTT,FMCG
75,TMPV,600.00,-2.86,-29.60,74.67,22.98,118755.0,-66858.0,159040.0,-26.74,34.0,X-LC,3.42,2.0,-0.56,1.14,0.03,XY24,NTT,AUTO
73,TCS,4311.58,2.27,-11.83,34.46,18.55,114923.0,-44740.0,333497.0,-23.95,54.0,X-LC,1.68,3.0,-0.39,2.40,13.46,X40,ATH,IT
81,VBL,671.64,-0.75,0.32,35.23,35.67,111631.0,1021.0,316863.0,-11.55,56.0,X-LC,2.96,5.0,0.01,2.28,14.22,X40N,ATH,FMCG
41,INFY,1972.00,-0.62,14.33,17.42,34.24,62313.0,44824.0,357712.0,-10.98,63.0,X-LC,7.48,6.0,0.72,2.57,23.79,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.46,-29.90,89.58,32.89,48480.0,-23088.0,54119.0,-768.50,72.0,L-MC,9.59,259.0,-0.48,0.39,49.04,XR,NTT,BANKS
13,BSOFT,831.70,-1.82,-22.91,95.60,50.79,103655.0,-32227.0,108426.0,-1.04,50.0,H-SC,9.78,171.0,-0.31,0.78,26.24,XR,ATH,IT
39,INDIGOPNTS,1408.00,-3.49,-15.24,18.00,0.02,26632.0,-26602.0,147957.0,125.45,49.0,M-SC,9.37,240.0,-1.00,1.06,28.70,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.13,5.85,12.63,19.23,20115.0,8806.0,159266.0,24.27,68.0,L-LC,10.45,182.0,0.44,1.14,28.37,XY25,ATH,MINING
36,ICICIPRULI,790.00,-2.18,9.75,19.11,30.73,41320.0,19215.0,216220.0,-15.87,51.0,X-MC,7.55,79.0,0.47,1.55,23.77,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.48,-28.32,57.50,12.90,90059.0,-61872.0,156624.0,130.77,60.0,H-SC,8.60,173.0,-0.69,1.13,30.23,OX40N,NTT,FINANCE
40,INDUSINDBK,1800.00,-0.46,-29.90,89.58,32.89,48480.0,-23088.0,54119.0,-768.50,72.0,L-MC,9.59,259.0,-0.48,0.39,49.04,XR,NTT,BANKS
17,COALINDIA,484.83,-0.13,5.85,12.63,19.23,20115.0,8806.0,159266.0,24.27,68.0,L-LC,10.45,182.0,0.44,1.14,28.37,XY25,ATH,MINING
57,RECLTD,446.00,0.30,0.43,20.22,20.74,41259.0,880.0,204050.0,33.49,57.0,M-MC,5.16,189.0,0.02,1.47,11.09,XY25,NTT,FINANCE
62,SATIN,274.00,-0.20,-16.60,82.06,51.83,189079.0,-45868.0,230416.0,-25.10,62.0,H-SC,3.91,148.0,-0.24,1.66,12.25,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.63
1,25,44.79
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.63
MC    30.63
LC    26.77
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.17
X40      24.18
X40N     12.88
XY25     10.87
XR        8.87
AR        8.40
OX40N     7.71
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.25
X-LC    23.34
H-SC    22.82
M-SC    11.12
X-SC     7.91
H-MC     4.52
M-MC     1.47
M-LC     1.24
L-LC     1.14
H-LC     1.05
L-SC     0.78
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.26,-9.70,45.69
IT,12.90,-19.22,78.78
FINANCE,11.12,-15.34,63.75
MISC,7.11,-31.25,84.93
ELECTRICAL,5.87,-14.79,56.84
PAINTS,5.82,-13.82,30.73
INSURANCE,4.68,-3.99,39.58
PHARMA,3.91,-7.97,41.96
BANKS,2.79,-37.55,120.89


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3328574.0,21
AR,1400872.0,10
XR,1256905.0,12
X40,1144409.0,15
X40N,951970.0,9
OX40N,721170.0,10
XY25,429510.0,7
SR,288936.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3618640.0,24
X-MC,1659815.0,16
M-SC,1511502.0,15
X-LC,966144.0,13
X-SC,818384.0,8
H-MC,449321.0,3
L-SC,182661.0,2
M-LC,119231.0,1
H-LC,86794.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1273417.0      6
           AR         938482.0      5
M-SC       XY24       866525.0      6
H-SC       XR         807947.0      7
X-MC       X40        546700.0      7
           XY24       463407.0      3
           X40N       419971.0      4
X-LC       X40        404014.0      6
X-SC       X40N       322182.0      3
M-SC       OX40N      318297.0      5
H-SC       OX40N      309858.0      4
X-SC       XY24       302507.0      3
H-SC       SR         288936.0      2
H-MC       AR         240517.0      2
X-MC       XY25       229737.0      2
X-LC       XY24       213914.0      2
           X40N       209817.0      2
H-MC       XY24       208804.0      1
X-SC       X40        193695.0      2
M-SC       XR         191601.0      2
X-LC       XY25       138399.0      3
M-SC       AR         135079.0      2
M-LC       XR         119231.0      1
L-SC       OX40N       93015.0      1
           XR          89646.0      1
H-LC       AR          86794.0      1
L-MC       XR          48480.0      1
M-MC       XY25        41259.0      1
L-LC       XY25        20115.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
